# BPSD: Transfer score-based annotations to individual audio versions

Johannes Zeitler (johannes.zeitler@audiolabs-erlangen.de), 2024

In [1]:
import os
import numpy as np
import scipy
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
data_dir = os.path.join("../", "2_Annotations")

In [3]:
dir_syncinfo = os.path.join(data_dir, "ann_audio_syncInfo")

In [4]:
files = [f for f in os.listdir(dir_syncinfo) if ".csv" in f]
files.sort()

In [5]:
annotations = ["localkey", "chord", "structureCoarse", "structureFine"]

for annotation in annotations:
    os.makedirs(os.path.join(data_dir, "ann_audio_"+annotation), exist_ok=True)

In [ ]:
for file in tqdm(files):
    piece = "_".join(file.split("_")[:-1])
    performer = file.split("_")[-1].split(".")[0]
    
    syncinfo = pd.read_csv(os.path.join(dir_syncinfo, file), sep=";")
    
    meas_to_time = scipy.interpolate.interp1d(syncinfo.measure, 
                                              syncinfo.time, 
                                              kind='linear', 
                                              bounds_error=False, 
                                              fill_value="extrapolate")
    
    for annotation in annotations:
        ann_in = pd.read_csv(os.path.join(data_dir, "ann_score_"+annotation, piece+".csv"), sep=";")

        ann_in.end = np.clip(ann_in.end, min(syncinfo.measure), max(syncinfo.measure))
        ann_in.start = np.clip(ann_in.start, min(syncinfo.measure), max(syncinfo.measure))

        
        ann_in.start = meas_to_time(ann_in.start)
        ann_in.end = meas_to_time(ann_in.end)
        
        if "duration" in ann_in.keys():
            ann_in.duration = ann_in.end - ann_in.start
        
        ann_in.to_csv(os.path.join(data_dir, "ann_audio_"+annotation, file), sep=";", header=True, index=False, float_format="%07.03f")